<a href="https://colab.research.google.com/github/Nid989/Isometric-Multi-task-NMT/blob/main/data_merger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets

In [ ]:
import datasets
import os
from tqdm.notebook import tqdm
import pandas as pd
import ast
from collections import defaultdict

tqdm.pandas()

In [ ]:
source = "en"
target = "de"

In [ ]:
language = f"{target}-{source}"
wmt_dataset = datasets.load_dataset("wmt16", language)

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# create output directory
current_directory = os.getcwd()
output_folder = f"wmt-{target}-{source}"
if not os.path.exists(os.path.join(current_directory, output_folder)):
    os.mkdir(output_folder)
path_to_output_folder = os.path.join(current_directory, output_folder)

In [ ]:
def normalize_data(df: pd.DataFrame, path_to_file: str):
  output = df.progress_apply(
      lambda row: ast.literal_eval(row.to_dict()['translation']),
      axis=1
  )
  pd_format_dict = defaultdict(list)
  for instance in tqdm(output, total=len(output)):
    pd_format_dict[source].append(instance[source])
    pd_format_dict[target].append(instance[target])

  df = pd.DataFrame(pd_format_dict)
  df.to_csv(path_to_file, index=False)

In [ ]:
data_types = ['train', 'validation', 'test']
for data_type in tqdm(data_types, total=3):
  path_to_file = os.path.join(path_to_output_folder, f"{data_type}.csv")
  wmt_dataset[data_type].to_csv(path_to_file, index=False)
  # read data
  wmt_df = pd.read_csv(path_to_file)
  # apply normalization
  normalize_data(wmt_df, path_to_file)

  0%|          | 0/3 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/455 [00:00<?, ?ba/s]

  0%|          | 0/4548885 [00:00<?, ?it/s]

  0%|          | 0/4548885 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:02<?, ?ba/s]

  0%|          | 0/2169 [00:03<?, ?it/s]

  0%|          | 0/2169 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2999 [00:00<?, ?it/s]

  0%|          | 0/2999 [00:00<?, ?it/s]

In [ ]:
# check for saved files
print(os.listdir(path_to_output_folder))

['train.csv', 'validation.csv', 'test.csv']


In [ ]:
mustc_dataset = datasets.load_dataset("enimai/MuST-C-de")

Using custom data configuration enimai--MuST-C-de-11a65737a7d9f57f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/enimai--MuST-C-de-11a65737a7d9f57f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# create output directory
current_directory = os.getcwd()
output_folder = f"mustc-{target}-{source}"
if not os.path.exists(os.path.join(current_directory, output_folder)):
    os.mkdir(output_folder)
path_to_output_folder = os.path.join(current_directory, output_folder)

In [ ]:
data_types = ['train', 'validation', 'test']
for data_type in tqdm(data_types, total=3):
  path_to_file = os.path.join(path_to_output_folder, f"{data_type}.csv")
  mustc_dataset[data_type].to_csv(path_to_file, index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# check for saved files
print(os.listdir(path_to_output_folder))

['train.csv', 'validation.csv', 'test.csv']


In [ ]:
# create output directory
current_directory = os.getcwd()
output_folder = f"final-{target}-{source}"
if not os.path.exists(os.path.join(current_directory, output_folder)):
    os.mkdir(output_folder)
path_to_output_folder = os.path.join(current_directory, output_folder)

In [ ]:
# merge both dataset
current_directory = os.getcwd()
wmt_folder_name = f"wmt-{target}-{source}"
mustc_folder_name = f"mustc-{target}-{source}"
path_to_wmt_folder = os.path.join(current_directory, wmt_folder_name)
path_to_mustc_folder = os.path.join(current_directory, mustc_folder_name)
data_types = ['train', 'validation', 'test']

for data_type in tqdm(data_types, total=3):
  filename = f"{data_type}.csv"
  path_to_wmt_datafile = os.path.join(path_to_wmt_folder, filename)
  path_to_mustc_datafile = os.path.join(path_to_mustc_folder, filename)
  wmt_df = pd.read_csv(path_to_wmt_datafile, lineterminator='\n')
  mustc_df = pd.read_csv(path_to_mustc_datafile, lineterminator='\n')
  df = pd.concat([wmt_df, mustc_df])
  df = df.sample(frac=1)
  print(df.shape)
  path_to_file = os.path.join(path_to_output_folder, filename)
  df.to_csv(path_to_file, index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

(4778588, 2)
(3592, 2)
(5640, 2)
